# Linearity test using sky background - SITCOM-510

This notebook contains the execution of the linearity tests using sky background described in https://jira.lsstcorp.org/browse/SITCOM-510.
                
This notebook is organized in  sections:

    1.1 Setup
    1.2 Conditions assessment
    1.3. Choose target from list of targets in ticket. 
    1.4. CWFS nearby
    1.5. Slew to target object.
    1.6  Perform the data acquisition sequence. 

-----
## Setup

### Import libraries

In [3]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
from astropy.time import Time
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

ImportError: cannot import name 'salobj' from 'lsst.ts' (unknown location)

### Setting up logger

In [ ]:
logger = logging.getLogger("SITCOM-510")
logger.level = logging.DEBUG

### Getting unique index for script

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index = os.getuid() * 10 + np.random.randint(0, 9)

logger.info(f'The generated index is {index}')

### Instantiate CWFS Script

In [ ]:
script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

### Forward ATCS and LATISS

In [ ]:
atcs = script.atcs
latiss = script.latiss

### Set up script log level

In [ ]:
script.log.level = logging.DEBUG

### Write start info into EFD

In [ ]:
script.log.info(f'START- SITCOM-510 Linearity test using sky background -- at {Time.now()}')

----
## Assess that conditions meet criteria
This must be performed during morning twilight, and requires photometric conditions.

Best to do this with fixed shutter-open time so we don’t introduce any shutter artifacts. Also best to do this during morning twilight so that any dome seeing has settled down and seeing is stable. We’d like fields that are reasonably dense with stars.

If we begin this entire operation at around 15 degree morning twilight then we should be able to run until sun is something like (this is a guess) 8-10 degrees below horizon, so we should get 20-30 minutes of data from this. That should be enough for 10 cycles through the data acquisition loop.  

----
## Choose target: Fields need to be reasonably dense with stars. Fields appropriate for this, for morning twilight for each month of the year, are listed in Table 1 of the PDF attached in https://jira.lsstcorp.org/browse/SITCOM-510

For November, we will use LinTest0513: Edge of NGC 1851. Move E
for Denser, W for sparser. 269 stars in 6 arcmin box. Extinction g 0.13

ra = 05:13:36.52
dec  = -40:02:44.2 

### Declare target and filter

In [ ]:
target = "LinTest0513"

In [ ]:
ra = '05:13:36.52'
dec = '-40:02:44.2'

In [ ]:
filter_to_use = 'SDSSr_65mm'

------
## CWFS
A CWFS is recommended just before the test to ensure the system is in focus. Based on the target, choose a nearby target to perform CWFS on. 

### Declare CWFS target 

Query for a source around the same area of the sky as the target source.

In [ ]:
cwfs_az = 110
cwfs_el = 60
cwfs_mag_limit = 8

In [ ]:
cwfs_target = await script.atcs.find_target(az=cwfs_az, el=cwfs_el, mag_limit=cwfs_mag_limit)
logger.info(f'Target for CWFS with magnitude limit {cwfs_mag_limit} is {cwfs_target}')

### Slew to the CWFS target

In [ ]:
await script.atcs.slew_object(cwfs_target, rot_type=RotType.PhysicalSky)

Uncomment the following line to take a snapshot to verify the target is positioned as expected 

In [ ]:
# exposure = await latiss.take_acq(
#         exptime=5, n=1, filter=filter_to_use, grating='empty_1', reason='Acquisition', program="SITCOM-510")
# logger.info(f'Acquisition exposure is {exposure}')

To offset the telescope and center the source (if required) uncomment the following line. <br>
Offsets are in detector X/Y coordinates and in arcseconds. 

In [ ]:
# await script.atcs.offset_xy(x=20, y=20)

### Set up configuration

In [ ]:
configuration = yaml.safe_dump({"filter": filter_to_use, 
                                "grating": 'empty_1',
                                "exposure_time": 20,
                                "program" : "SITCOM-510"})

The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included.  
Therefore, it is included here despite being a non-operation in the first instance.  

In [ ]:
await script.set_state(ScriptState.UNCONFIGURED)

### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images.

In [ ]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

### Set groupID and launch the script
This sets the same group ID for all exposures taken in the script.

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
    groupId=astropy.time.Time.now().isot)

await script.do_setGroupId(group_id_data)
await script.arun()

### Stop tracking: If required, then uncomment and use the following cell to stop the telescope from tracking, but you will lose your acquisition.

In [ ]:
# await script.atcs.stop_tracking()

-------
## Slew to target object

### Confirm that the target is correct

In [ ]:
logger.info(f'Target is {target}')

### Slew to the target object

In [ ]:
#await script.atcs.slew_object(target, rot_type=RotType.PhysicalSky)
await script.atcs.slew.icrs(

To offset the telescope and move the source (if required) uncomment the following line. <br>
Offsets are in detector X/Y coordinates and in arcseconds. 

In [ ]:
await script.atcs.offset_xy(x=20, y=20)

----
## Data Acquisition Sequence: Once sky brightness level reaches 1000 ADU above bias level in a red band (SDSSr) in a 10 second exposure, begin the following sequence, to be looped until sky reaches saturation in a 10 second image:  

###  Check signal level: 
Take a 10-s image to check that the sky brightness is around 1000 ADU above bias level. To check signal levels, use the new CCS feature http://ccs.lsst.org/RecentImages/auxtel.html. As you hover over the image, the pixel and counts are displayed on the bottom right. 

In [ ]:
sky_brightness_test = await latiss.take_engtest(
        exptime=10, n=1, filter=filter_to_use, grating='empty_1', reason='Sky_Brightness_Level_Test', program = "SITCOM-510")
logger.info(f'Sky Brightness test exposure is {sky_brightness_test}')

### Data Acquisition Loop
For time=start:end
Take 3 x 5 sec dark images (to test for persistence as well)
Take 3 x 5 sec exposures
Take 3 x 10 sec exposures
Take 3 x 30 sec exposures
Iterate

In [ ]:
while i<12:
    logger.info(f'Loop cycle number {i}')
    darks_5sec = await latiss.take_darks(
        exptime=5, n=3, filter=filter_to_use, grating='empty_1', reason='5sec_Darks_Linearity_using_sky_background', program ="SITCOM-510")
    logger.info(f'5sec Darks in {i} loop are {darks_5sec}')
    exp_5sec = await latiss.take_image(
        exptime=5, n=3, filter=filter_to_use, grating='empty_1', reason='5sec_Images_Linearity_using_sky_background', program ="SITCOM-510")
    logger.info(f'5sec images in {i} loop are {exp_5sec}')
    exp_10sec = await latiss.take_image(
        exptime=5, n=3, filter=filter_to_use, grating='empty_1', reason='10sec_Images_Linearity_using_sky_background', program ="SITCOM-510")
    logger.info(f'10sec images in {i} loop are {exp_10sec}')
    exp_30sec = await latiss.take_image(
        exptime=30, n=3, filter=filter_to_use, grating='empty_1', reason='30sec_Images_Linearity_using_sky_background', program ="SITCOM-510")
    logger.info(f'30sec images in {i} loop are {exp_30sec}')
    i +=1 

### Going intra focus with -z_offset

In [ ]:
await script.atcs.rem.ataos.cmd_offset.set_start(z=-2*z_offset)

### Data Acquisition Intra Focus

In [ ]:
intra_images = await latiss.take_focus(
    exptime=30, n=5, filter=filter_to_use, grating='empty_1', reason='Large_Donuts_intra', program ="SITCOM-510")
logger.info(f'Intra-Focus images are {intra_images}')

### Move hexapod back to in-focus (zero-offset) position

In [ ]:
await script.atcs.rem.ataos.cmd_offset.set_start(z=z_offset)

In [ ]:
script.log.info(f'END- SITCOM-510 Linearity test using sky background -- at {Time.now()}')